In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def cell_fn(cell_val):
    return pd.Series([v.split('-')[0] for v in cell_val])

drop_col_ids = np.arange(5, 32)

scores_ramin = pd.read_csv('/home/srivathsa/projects/studies/gad/reader_study/mrm_reader1.csv')
scores_ramin.drop(scores_ramin.columns[drop_col_ids], axis=1, inplace=True)
scores_ramin = scores_ramin.apply(cell_fn, axis=1)
scores_ramin.columns = ['case', 'tumor_cateogry', 'iq_score', 'qc_score', 'vessel']

scores_greg = pd.read_csv('/home/srivathsa/projects/studies/gad/reader_study/mrm_reader2.csv')
scores_greg.drop(scores_greg.columns[drop_col_ids], axis=1, inplace=True)
scores_greg = scores_greg.apply(cell_fn, axis=1)
scores_greg.columns = ['case', 'tumor_cateogry', 'iq_score', 'qc_score', 'vessel']

In [6]:
dict_cmap = json.loads(open('/home/srivathsa/projects/studies/gad/reader_study/case_map.json', 'r').read())

In [7]:
case_pairs = []

for case_dict in dict_cmap:
    if 'SubtleGad' in case_dict['Patient ID']:
        continue
    pair_obj = {
        'patient_id': case_dict['Patient ID'],
        'gt_case': case_dict['Case Number']
    }
    
    gad_obj = [
        d for d in dict_cmap 
        if 'SubtleGad' in d['Patient ID'] and d['Patient ID'].replace('SubtleGad:', '') == case_dict['Patient ID']
    ][0]
    pair_obj['gad_case'] = gad_obj['Case Number']
    case_pairs.append(pair_obj)

In [8]:
df_ref = scores_ramin

for pair_dict in case_pairs:
    gt_row = df_ref.loc[df_ref['case'] == pair_dict['gt_case']].values[0]
    gt_tumor_class = int(gt_row[1])
    
    gad_row = df_ref.loc[df_ref['case'] == pair_dict['gad_case']].values[0]
    gad_tumor_class = int(gad_row[1])
    
    if gt_tumor_class != gad_tumor_class:
        print(pair_dict)
        print(gt_tumor_class, gad_tumor_class)
    
    

{'patient_id': 'Patient_0157', 'gt_case': 'Case0005', 'gad_case': 'Case0080'}
2 1
{'patient_id': 'Prisma3', 'gt_case': 'Case0009', 'gad_case': 'Case0021'}
1 2
{'patient_id': 'Patient_0486', 'gt_case': 'Case0018', 'gad_case': 'Case0051'}
1 2
{'patient_id': 'NO130', 'gt_case': 'Case0037', 'gad_case': 'Case0057'}
0 1
{'patient_id': 'Patient_0333', 'gt_case': 'Case0060', 'gad_case': 'Case0085'}
1 2
{'patient_id': 'Patient_0535', 'gt_case': 'Case0062', 'gad_case': 'Case0049'}
0 1
{'patient_id': 'NO71', 'gt_case': 'Case0072', 'gad_case': 'Case0063'}
1 0
{'patient_id': 'NO26', 'gt_case': 'Case0098', 'gad_case': 'Case0020'}
1 0
{'patient_id': 'NO6', 'gt_case': 'Case0099', 'gad_case': 'Case0030'}
1 2


In [33]:
df_ref = scores_ramin
miscount = 0

for pair_dict in case_pairs:
    gt_row = df_ref.loc[df_ref['case'] == pair_dict['gt_case']].values[0]
    gt_vess = int(gt_row[4])
    
    gad_row = df_ref.loc[df_ref['case'] == pair_dict['gad_case']].values[0]
    gad_vess = int(gad_row[4])
    
    if gt_vess != gad_vess:
        print(gt_vess, gad_vess, pair_dict)
    
    a = int(gad_row[2])
    b = int(gad_row[4])
    if a == b or abs(a-b) ==1:
        miscount += 1

4 2 {'patient_id': 'Patient_0157', 'gt_case': 'Case0005', 'gad_case': 'Case0080'}
4 2 {'patient_id': 'Patient_0486', 'gt_case': 'Case0018', 'gad_case': 'Case0051'}
3 2 {'patient_id': 'Patient_0178', 'gt_case': 'Case0022', 'gad_case': 'Case0033'}
2 5 {'patient_id': 'NO67', 'gt_case': 'Case0023', 'gad_case': 'Case0086'}
3 1 {'patient_id': 'NO130', 'gt_case': 'Case0037', 'gad_case': 'Case0057'}
5 2 {'patient_id': 'Prisma1', 'gt_case': 'Case0041', 'gad_case': 'Case0074'}
5 4 {'patient_id': 'Prisma9', 'gt_case': 'Case0052', 'gad_case': 'Case0042'}
5 2 {'patient_id': 'Prisma22', 'gt_case': 'Case0053', 'gad_case': 'Case0031'}
4 5 {'patient_id': 'NO120', 'gt_case': 'Case0055', 'gad_case': 'Case0003'}
3 4 {'patient_id': 'Patient_0400', 'gt_case': 'Case0058', 'gad_case': 'Case0071'}
3 5 {'patient_id': 'Patient_0172', 'gt_case': 'Case0059', 'gad_case': 'Case0076'}
3 4 {'patient_id': 'Patient_0087', 'gt_case': 'Case0061', 'gad_case': 'Case0010'}
2 5 {'patient_id': 'Patient_0535', 'gt_case': 'Case0

In [17]:
df_ref = scores_greg
eq_count = 0
grt_count = 0
less_count = 0

dict_iq = {}

for pair_dict in case_pairs:
    gt_row = df_ref.loc[df_ref['case'] == pair_dict['gt_case']].values[0]
    gt_iq = int(gt_row[2])
    
    gad_row = df_ref.loc[df_ref['case'] == pair_dict['gad_case']].values[0]
    gad_iq = int(gad_row[2])
    
    if gt_iq == gad_iq:
        eq_count += 1
    if gad_iq > gt_iq:
        grt_count += 1
    if gad_iq < gt_iq:
#         print(gad_iq, gt_iq)
        less_count += 1
    
    if gad_iq in dict_iq:
        dict_iq[gad_iq] += 1
    else:
        dict_iq[gad_iq] = 1

In [18]:
dict_iq

{5: 11, 3: 11, 2: 2, 4: 24, 1: 2}

In [21]:
uniq_cases = []
for index, row in scores_ramin.iterrows():
    tumor_score = int(row.values[1])
    greg_row = scores_greg.loc[scores_greg['case'] == row.values[0]].values[0]
    tumor_score_greg = int(greg_row[1])
    case = [d for d in dict_cmap if d['Case Number'] == row.values[0]][0]
    if tumor_score == tumor_score_greg and tumor_score == 1:
#         print(row.values[0], case['Patient ID'])
        uniq_cases.append(case['Patient ID'].replace('SubtleGad:', ''))
# sorted(set(uniq_cases))

In [33]:
cases = ['NO108','NO113','NO120','NO129','NO18','NO26','NO54','NO56','NO6','NO67','NO71',
         'NO79','Patient_0087','Patient_0090','Patient_0134','Patient_0172','Patient_0173',
         'Patient_0255','Patient_0333','Patient_0342','Patient_0353','Patient_0408','Patient_0486',
         'Patient_0526','Patient_0535','Prisma3','Prisma4','Prisma6','Prisma9']

for case in cases:
    case_rows = [d for d in dict_cmap if case in d['Patient ID']]
    gad_case = case_rows[0] if 'SubtleGad:' in case_rows[0]['Patient ID'] else case_rows[1]
    gt_case = case_rows[1] if 'SubtleGad:' in case_rows[0]['Patient ID'] else case_rows[0]
    
    greg_gt_score = scores_greg[scores_greg['case'] == gt_case['Case Number']].values[0][1]
    greg_gad_score = scores_greg[scores_greg['case'] == gad_case['Case Number']].values[0][1]
    
    ramin_gt_score = scores_ramin[scores_ramin['case'] == gt_case['Case Number']].values[0][1]
    ramin_gad_score = scores_ramin[scores_ramin['case'] == gad_case['Case Number']].values[0][1]
    
    print('{},{},{},{},{}'.format(case, greg_gt_score, ramin_gt_score, greg_gad_score, ramin_gad_score))

NO108,1,1,1,1
NO113,1,1,1,1
NO120,1,1,1,1
NO129,1,1,1,1
NO18,1,1,1,1
NO26,1,1,0,0
NO54,1,1,1,1
NO56,1,1,1,1
NO6,2,2,2,2
NO67,1,1,1,1
NO71,1,1,1,0
NO79,1,1,1,1
Patient_0087,1,1,1,1
Patient_0090,1,1,1,1
Patient_0134,1,1,1,1
Patient_0172,1,1,1,1
Patient_0173,1,1,1,1
Patient_0255,1,1,1,1
Patient_0333,1,1,1,2
Patient_0342,1,1,1,1
Patient_0353,1,1,0,1
Patient_0408,1,1,1,1
Patient_0486,1,1,1,2
Patient_0526,1,1,1,1
Patient_0535,0,0,1,1
Prisma3,1,1,2,2
Prisma4,1,1,1,1
Prisma6,1,1,1,1
Prisma9,1,1,1,1
